## analyzer

In [2]:
import ast
from pprint import pprint

def analyze(code):
    tree = ast.parse(code)
    analyzer = Analyzer()
    analyzer.visit(tree)
    analyzer.report()


class Analyzer(ast.NodeVisitor):
    def __init__(self):
        self.stats = {"variable": []}
        
    def visit_Name(self, node):
        # self.stats["variable"].append(node.id)
        self.generic_visit(node)
    
    def visit_Assign(self, node):
        for target in node.targets:
            if(type(target) == ast.Name):
                self.stats["variable"].append(target.id)
            if(type(target) == ast.Subscript):
                self.stats["variable"].append(target.value.id)
        self.generic_visit(node)
        
    def visit_Expr(self, node):
        if(type(node.value)==ast.Call and type(node.value.func) == ast.Attribute):
            self.stats["variable"].append(node.value.func.value.id)
        self.generic_visit(node)

    def report(self):
        pprint(self.stats)

## private cell

In [32]:
from IPython.core.magic import (register_line_magic, register_cell_magic)
import copy
import re

@register_cell_magic
def private(line, cell):
    name=line.split(" ")[0]
    variables=line.split(" ")[1:]
    
    content = f'class {name}:\n'
    for variable in variables:
        content +=f'\t{variable}=copy.deepcopy({variable})\n'
    for line in cell.split('\n'):
        pattern=r'def .*\(.*\):'
        match = re.search(pattern, line)
        if(match):
            l = line.split('(')
            if(l[1][0]==')'):
                newl = l[0] + '(self'+ "".join(l[1:])
            else:
                newl = l[0] + '(self,' + "".join(l[1:])
            line = newl
        content += f'\t{line}\n'
    content += f'{name}={name}()'
    print(content)
    exec(content, globals())

@register_cell_magic
def privateMain(line, cell):
    name=line.split(" ")[0]
    variables=line.split(" ")[1:]

    content = f'class {name}:\n'
    for variable in variables:
        content +=f'\t{variable}=copy.deepcopy({variable})\n'
    for line in cell.split('\n'):
        pattern=r'def .*\(.*\):'
        match = re.search(pattern, line)
        if(match):
            l = line.split('(')
            if(l[1][0]==')'):
                newl = l[0] + '(self'+ "".join(l[1:])
            else:
                newl = l[0] + '(self,' + "".join(l[1:])
            line = newl
        content += f'\t{line}\n'
    content += f'{name}={name}()'
    print(content)
    exec(content, globals())
    for variable in variables:
        exec(f'{variable} = {name}.{variable}', globals())
        print(f'{variable} = {name}.{variable}')

In [25]:
test = 1

In [27]:
code = """
test = 'April'
"""

In [28]:
analyze(code)

{'variable': ['test']}


In [29]:
%%private april test
test = 'April'

class april:
	test=copy.deepcopy(test)
	test = 'April'
	
april=april()


In [33]:
%%privateMain april test
test = 'April'

class april:
	test=copy.deepcopy(test)
	test = 'April'
	
april=april()
test = april.test
